# Columns Infos.

* **Id** - an Id that represents a (Store, Date) duple within the test set
* **Store** - a unique Id for each store
* **Sales** - the turnover for any given day (this is what you are predicting)
* **Customers** - the number of customers on a given day
* **Open** - an indicator for whether the store was open: 0 = closed, 1 = open
* **StateHoliday** - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
* **SchoolHoliday** - indicates if the (Store, Date) was affected by the closure of public schools
* **StoreType** - differentiates between 4 different store models: a, b, c, d
* **Assortment** - describes an assortment level: a = basic, b = extra, c = extended
* **CompetitionDistance** - distance in meters to the nearest competitor store
* **CompetitionOpenSince[Month/Year]** - gives the approximate year and month of the time the nearest competitor was opened
* **Promo** - indicates whether a store is running a promo on that day
* **Promo2** - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
* **Promo2Since[Year/Week]** - describes the year and calendar week when the store started participating in Promo2
* **PromoInterval** - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store.

# 0.0 Imports

In [23]:
import pandas as pd
import inflection
import math

## 0.1 Helper Functions

## 0.2 Loading Data

In [24]:
df_sales_raw = pd.read_csv('C:\\Users\\felip\\repos\\data_science_em_producao\\rossmann_data\\train.csv', low_memory = False)
df_store_raw = pd.read_csv('C:\\Users\\felip\\repos\\data_science_em_producao\\rossmann_data\\store.csv', low_memory = False)

### [Merge - join two DataFrames.](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)

In [25]:
df_raw = pd.merge(df_sales_raw, df_store_raw, how = 'left', on = 'Store')

# 1.0 Data Description

## 1.1 Rename Columns

In [26]:
# Dataset copy
df1 = df_raw.copy()

In [27]:
cols_old = list(df1.columns)

In [28]:
# Transforming columns names from CamelCase to SnakeCase.

snakecase = lambda x: inflection.underscore(x)

cols_new = list(map(snakecase, cols_old))

# Rename columns
df1.columns = cols_new

In [29]:
df1.columns

Index(['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday', 'store_type', 'assortment',
       'competition_distance', 'competition_open_since_month',
       'competition_open_since_year', 'promo2', 'promo2_since_week',
       'promo2_since_year', 'promo_interval'],
      dtype='object')

## 1.2 Data Dimensions

In [30]:
print(f'Number of Rows: {df1.shape[0]}.\nNumber of Columns: {df1.shape[1]}.')

Number of Rows: 1017209.
Number of Columns: 18.


## 1.3 Data Types

In [31]:
df1.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

In [32]:
df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

## 1.4 Check NaN

In [33]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

## 1.5 Fillout NaN

The first method used, marked as a comment, was the method used by the instructor,
however, the "fillna ()" method is more efficient.

In [38]:
# competition_distance
    # I've assumed that the missing values on "competition_distance" columns means that there are no competition 
    # near by the store, so I put a huge aleatory distance (bigger then oririnal max distance) distance. 
    
# df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000 if math.isna(x) else x)

df1['competition_distance'].fillna(200000, inplace = True)

# competition_open_since_month

# df1['competition_open_since_month'] = df1['competition_open_since_month'].apply(lambda x: x['date].month 
    # if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'])
    
df1['competition_open_since_month'].fillna(df1['date'].dt.month, inplace = True)

#competition_open_since_year  

# df1['competition_open_since_year'] = df1['competition_open_since_year'].apply(lambda x: x['date].year 
    # if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'])
    
df1['competition_open_since_year'].fillna(df1['date'].dt.year, inplace = True)

#promo2_since_week   

#df1['promo2_since_week'] = df1['promo2_since_week'].apply(lambda x: x['date].week 
    # if math.isnan(x['promo2_since_week']) else x['promo2_since_week'])

df1['promo2_since_week'].fillna(df1['date'].dt.week, inplace = True)

#promo2_since_year     

#df1['promo2_since_year'] = df1['promo2_since_year'].apply(lambda x: x['date].year 
    # if math.isnan(x['promo2_since_year']) else x['promo2_since_year'])

df1['promo2_since_year'].fillna(df1['date'].dt.year, inplace = True)

#promo_interval

month_map = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
             7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

df1['promo_interval'].fillna(0, inplace = True)

df1['month_map'] = df1['date'].dt.month.map(month_map)

df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis = 1)

<ipython-input-38-62aa0e348fdd>:28: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df1['promo2_since_week'].fillna(df1['date'].dt.week, inplace = True)


In [42]:
## 1.6 

store                           0
day_of_week                     0
date                            0
sales                           0
customers                       0
open                            0
promo                           0
state_holiday                   0
school_holiday                  0
store_type                      0
assortment                      0
competition_distance            0
competition_open_since_month    0
competition_open_since_year     0
promo2                          0
promo2_since_week               0
promo2_since_year               0
promo_interval                  0
month_map                       0
is_promo                        0
dtype: int64